# Street Fighter 6 Frame Extraction Test

This notebook provides functionality to:
- Extract frames from MP4 video files at specified intervals
- Save extracted frames as PNG images with timestamp naming
- Organize output in structured folder hierarchy
- Support custom frame extraction rates

The extracted frames can then be analyzed using the test-export notebook
or other image analysis tools.

Reproduit le comportement du script `test/test-extract.py`

In [ ]:
import sys
import os
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Add the project root to the Python path
project_root = os.path.abspath('../..')
if project_root not in sys.path:
    sys.path.append(project_root)

# Vérifier si frame_extractor.py existe dans le répertoire racine ou dans src/
try:
    from frame_extractor import FrameExtractor
    print("✅ FrameExtractor importé depuis la racine du projet")
except ImportError:
    try:
        from src.frame_extractor import FrameExtractor
        print("✅ FrameExtractor importé depuis src/")
    except ImportError:
        print("❌ Erreur: FrameExtractor non trouvé. Vérifiez que le fichier frame_extractor.py existe.")
        print("Fichiers disponibles dans le répertoire racine:")
        root_files = [f for f in os.listdir('../..') if f.endswith('.py')]
        for f in root_files:
            print(f"  - {f}")
        print("Fichiers disponibles dans src/:")
        src_files = [f for f in os.listdir('../../src') if f.endswith('.py')]
        for f in src_files:
            print(f"  - {f}")

## Configuration

In [ ]:
# Configuration
VIDEO_PATH = 'input/match.mp4'  # Changez ce chemin selon votre vidéo à traiter
FOLDER_NAME = None  # None pour utiliser le nom automatique, ou spécifiez un nom
NO_PROMPT = True  # True pour utiliser le nom automatique sans demander
DEBUG = True  # Active le mode debug

print(f"Vidéo à traiter: {VIDEO_PATH}")
print(f"Nom du dossier de sortie: {FOLDER_NAME or 'automatique'}")
print(f"Mode sans prompt: {NO_PROMPT}")
print(f"Mode debug: {DEBUG}")

## Vérification de l'existence du fichier vidéo

In [ ]:
if not os.path.exists(VIDEO_PATH):
    print(f"❌ Error: Video file '{VIDEO_PATH}' does not exist.")
    
    # Lister les vidéos disponibles
    input_dir = 'input'
    if os.path.exists(input_dir):
        video_extensions = ('.mp4', '.avi', '.mov', '.mkv', '.wmv', '.flv')
        available_videos = [f for f in os.listdir(input_dir) if f.lower().endswith(video_extensions)]
        if available_videos:
            print(f"Vidéos disponibles dans {input_dir}:")
            for video in available_videos:
                video_path = os.path.join(input_dir, video)
                size_mb = os.path.getsize(video_path) / (1024 * 1024)
                print(f"  - {video} ({size_mb:.1f} MB)")
        else:
            print(f"❌ Aucune vidéo trouvée dans {input_dir}")
    else:
        print(f"❌ Dossier {input_dir} non trouvé")
else:
    print(f"✅ Vidéo trouvée: {VIDEO_PATH}")
    size_mb = os.path.getsize(VIDEO_PATH) / (1024 * 1024)
    print(f"Taille du fichier: {size_mb:.1f} MB")

## Fonction d'extraction des frames

In [ ]:
def extract_frames_with_progress(video_path, folder_name=None, no_prompt=True, debug=True):
    """
    Extrait les frames d'une vidéo avec affichage de progression.
    """
    try:
        print(f"🎬 Initialisation de l'extracteur de frames...")
        extractor = FrameExtractor(video_path, folder_name, no_prompt, debug=debug)
        
        print(f"🚀 Début de l'extraction des frames...")
        result = extractor.extract_frames()
        
        print(f"✅ Extraction terminée avec succès!")
        return result
        
    except Exception as e:
        print(f"❌ Erreur lors de l'extraction: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

## Exécution de l'extraction

In [ ]:
if os.path.exists(VIDEO_PATH):
    print(f"🎯 Extraction des frames de: {VIDEO_PATH}")
    
    # Exécuter l'extraction
    result = extract_frames_with_progress(VIDEO_PATH, FOLDER_NAME, NO_PROMPT, DEBUG)
    
    if result is not None:
        print("\n=== RÉSUMÉ DE L'EXTRACTION ===")
        # Note: Le retour de la fonction dépend de l'implémentation de FrameExtractor
        # Ici on suppose qu'elle retourne des informations sur l'extraction
        
        # Vérifier le dossier de sortie créé
        # Le dossier de sortie est généralement dans input/frames/
        frames_base_dir = 'input/frames'
        if os.path.exists(frames_base_dir):
            subdirs = [d for d in os.listdir(frames_base_dir) if os.path.isdir(os.path.join(frames_base_dir, d))]
            if subdirs:
                print(f"Dossiers de frames créés dans {frames_base_dir}:")
                for subdir in sorted(subdirs, key=lambda x: os.path.getmtime(os.path.join(frames_base_dir, x)), reverse=True):
                    subdir_path = os.path.join(frames_base_dir, subdir)
                    frame_count = len([f for f in os.listdir(subdir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
                    print(f"  - {subdir}: {frame_count} frames")
                
                # Prendre le dossier le plus récent comme résultat de cette extraction
                latest_dir = sorted(subdirs, key=lambda x: os.path.getmtime(os.path.join(frames_base_dir, x)), reverse=True)[0]
                latest_dir_path = os.path.join(frames_base_dir, latest_dir)
                
                print(f"\n📁 Dossier de sortie le plus récent: {latest_dir_path}")
                
                # Compter les frames extraites
                extracted_frames = [f for f in os.listdir(latest_dir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                print(f"📸 Nombre de frames extraites: {len(extracted_frames)}")
                
                if extracted_frames:
                    print("Exemples de frames:")
                    for frame in extracted_frames[:5]:
                        print(f"  - {frame}")
                    if len(extracted_frames) > 5:
                        print(f"  ... et {len(extracted_frames) - 5} autres")
        
else:
    print(f"❌ Impossible d'extraire - la vidéo {VIDEO_PATH} n'existe pas")

## Visualisation des frames extraites (optionnel)

In [ ]:
# Optionnel: Afficher quelques frames extraites
frames_base_dir = 'input/frames'

if os.path.exists(frames_base_dir):
    subdirs = [d for d in os.listdir(frames_base_dir) if os.path.isdir(os.path.join(frames_base_dir, d))]
    
    if subdirs:
        # Prendre le dossier le plus récent
        latest_dir = sorted(subdirs, key=lambda x: os.path.getmtime(os.path.join(frames_base_dir, x)), reverse=True)[0]
        latest_dir_path = os.path.join(frames_base_dir, latest_dir)
        
        # Lister les frames
        frame_files = [f for f in os.listdir(latest_dir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        frame_files.sort()
        
        if frame_files:
            import cv2 as cv
            
            # Afficher les 4 premières frames
            num_to_show = min(4, len(frame_files))
            
            fig, axes = plt.subplots(2, 2, figsize=(15, 10))
            axes = axes.flatten()
            
            for i in range(num_to_show):
                frame_path = os.path.join(latest_dir_path, frame_files[i])
                frame = cv.imread(frame_path)
                if frame is not None:
                    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                    axes[i].imshow(frame_rgb)
                    axes[i].set_title(f'Frame: {frame_files[i]}')
                    axes[i].axis('off')
                else:
                    axes[i].text(0.5, 0.5, 'Erreur de lecture', ha='center', va='center')
                    axes[i].set_title(f'Erreur: {frame_files[i]}')
            
            # Masquer les axes non utilisés
            for j in range(num_to_show, 4):
                axes[j].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            print(f"Affichage de {num_to_show} frames sur {len(frame_files)} extraites")
        else:
            print("❌ Aucune frame trouvée dans le dossier le plus récent")
    else:
        print("❌ Aucun dossier de frames trouvé")
else:
    print(f"❌ Dossier de frames non trouvé: {frames_base_dir}")

## Informations sur les frames extraites

In [ ]:
# Afficher des statistiques détaillées sur l'extraction
frames_base_dir = 'input/frames'

if os.path.exists(frames_base_dir):
    print("=== STATISTIQUES D'EXTRACTION ===")
    
    all_subdirs = [d for d in os.listdir(frames_base_dir) if os.path.isdir(os.path.join(frames_base_dir, d))]
    
    if all_subdirs:
        print(f"Nombre total de dossiers d'extraction: {len(all_subdirs)}")
        
        total_frames = 0
        for subdir in all_subdirs:
            subdir_path = os.path.join(frames_base_dir, subdir)
            frame_count = len([f for f in os.listdir(subdir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            total_frames += frame_count
        
        print(f"Nombre total de frames extraites: {total_frames}")
        
        # Calculer l'espace disque utilisé
        total_size = 0
        for subdir in all_subdirs:
            subdir_path = os.path.join(frames_base_dir, subdir)
            for frame_file in os.listdir(subdir_path):
                frame_path = os.path.join(subdir_path, frame_file)
                if os.path.isfile(frame_path):
                    total_size += os.path.getsize(frame_path)
        
        print(f"Espace disque utilisé: {total_size / (1024*1024):.1f} MB")
        
        if total_frames > 0:
            avg_size = total_size / total_frames
            print(f"Taille moyenne par frame: {avg_size / 1024:.1f} KB")
    else:
        print("❌ Aucun dossier d'extraction trouvé")
else:
    print(f"❌ Dossier de base non trouvé: {frames_base_dir}")